In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
from sklearn.model_selection import train_test_split

In [78]:
traindata = pd.read_csv('datasets/upvotes/train_NIR5Yl1.csv')
traindata.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [79]:
testdata = pd.read_csv('datasets/upvotes/test_8i3B3FC.csv')
testdata.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [80]:
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141448 entries, 0 to 141447
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          141448 non-null  int64  
 1   Tag         141448 non-null  object 
 2   Reputation  141448 non-null  float64
 3   Answers     141448 non-null  float64
 4   Username    141448 non-null  int64  
 5   Views       141448 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 6.5+ MB


## Clean Data

In [81]:
testdata =testdata.dropna()
testdata.describe()

,ID,Reputation,Answers,Username,Views
count,141448.000000,1.414480e+05,141448.000000,141448.000000,1.414480e+05
mean,235743.073497,7.920927e+03,3.914873,81348.231117,2.984633e+04
std,136269.867118,2.791072e+04,3.577460,49046.098215,8.034374e+04
min,7.000000,0.000000e+00,0.000000,4.000000,9.000000e+00
25%,117797.000000,2.860000e+02,2.000000,40222.750000,2.608000e+03
50%,235830.000000,1.245000e+03,3.000000,78795.500000,8.977000e+03
75%,353616.000000,5.123000e+03,5.000000,122149.000000,2.698925e+04
max,471488.000000,1.042428e+06,73.000000,175737.000000,5.004669e+06


## Ordinal Encoder

In [82]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [83]:
tag_cat = traindata[['Tag']]
tag_cat_encoded = ordinal_encoder.fit_transform(tag_cat)
tag_cat_encoded[:10]

array([[0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [7.],
       [1.],
       [4.],
       [4.],
       [1.]])

## One-Hot Encoder

In [84]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
tag_cat_1hot = cat_encoder.fit_transform(tag_cat)
tag_cat_1hot.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Custom Transformer

In [85]:
class custom_transformer:
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        answer_index, views_index = 3,5
        answers_per_view = X[:, answer_index] / X[:, views_index]
        return np.c_[X, answers_per_view]


In [86]:
extra_cols = transform(traindata.values)

In [87]:
training_extra = pd.DataFrame(
    extra_cols,
    columns=list(traindata.columns)+["answers per view"],
    index=traindata.index)
training_extra.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,answers per view
0,52664,a,3942.0,2.0,155623,7855.0,42.0,0.000255
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0.000215
2,468453,c,1358.0,4.0,56177,8067.0,60.0,0.000496
3,96996,a,264.0,3.0,168793,27064.0,9.0,0.000111
4,131465,c,4271.0,4.0,112223,13986.0,83.0,0.000286


## Scale Views from 0 - 1

In [88]:
from sklearn import preprocessing
x = traindata[["Answers","Views"]] #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_scaled[:5]

array([[0.02631579, 0.00149989],
       [0.15789474, 0.01066555],
       [0.05263158, 0.00154042],
       [0.03947368, 0.005172  ],
       [0.05263158, 0.00267193]])

In [106]:
data = traindata.drop("Upvotes", axis = 1)
vals = traindata.Upvotes
traindata.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


## Grid search for optimization

In [107]:
from sklearn.ensemble import GradientBoostingRegressor
gra_reg =  GradientBoostingRegressor()

In [120]:
train, test = train_test_split(traindata, test_size=0.30)
train[:5]

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
307650,468144,c,854.0,3.0,83401,1512.0,20.0
121371,11923,c,859.0,4.0,88355,486.0,2.0
128897,250656,p,762.0,1.0,43920,9180.0,34.0
300177,222530,j,5806.0,6.0,55647,45352.0,383.0
94284,97696,j,215.0,5.0,46918,21520.0,18.0


In [122]:
data = train.drop("Upvotes", axis=1) # drop labels for training set
labels = train["Upvotes"].copy()

In [123]:
preped_data = full_pipeline.fit_transform(data)

In [125]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(preped_data, labels)

RandomForestRegressor(random_state=42)

In [128]:
from sklearn.metrics import mean_squared_error
predictions = forest_reg.predict(preped_data)
forest_mse = mean_squared_error(labels, predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

555.1864541731752

In [135]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(preped_data, labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [136]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 10}

## Pipeline for all data processing and Prediction

#### number pipeline

In [ ]:
traindata.head()

In [ ]:
traindata = traindata.drop("ID", axis = 1)
# drop ID because it provides no real information

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

train_num = traindata.drop(["Tag","Username"], axis=1)
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])
train_num_tr = num_pipeline.fit_transform(train_num)
train_num_tr[:5]

In [112]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import  RandomForestRegressor

num_attribs = list(train_num)
num_attribs.pop
num
cat_attribs = ["Tag"]


full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

prepared = full_pipeline.fit_transform(traindata)

In [117]:
print(num_attribs)

['Reputation', 'Answers', 'Views']


In [116]:
num_attribs.pop()

'Upvotes'